In [1]:
import requests
import shutil
import os
import io
import json
import time
import datetime
import dateutil
import random
import re
import ast
import pickle
import numpy as np
import pandas as pd
from IPython.display import clear_output
from pprint import pprint
import bs4
import matplotlib.pyplot as plt
import copy

parent_folder='steam_data'

csv_folder=os.path.join(parent_folder,"bulk","csv_no_text","early_access_standardized")
csv_english_folder=os.path.join(parent_folder,"bulk","csv_english","early_access_standardized")
testing_folder=os.path.join(parent_folder,"bulk","testing")

In [2]:
with open(os.path.join(testing_folder,'still_in_early_access_FINAL'),'r') as file:
    current_ea_games=sorted(list(set([int(x) for x in file.read().split('\n')[:-1]])))
with open(os.path.join(testing_folder,'out_of_early_access_FINAL'),'r') as file:
    former_ea_games=sorted(list(set([int(x) for x in file.read().split('\n')[:-1]])))
full_games_list=sorted(list(set(current_ea_games+former_ea_games)))
with open(os.path.join(parent_folder,'official_appid_json_list'),'r') as file:
    temp_json=json.load(file)['applist']['apps']
official_app_list={}
for x in temp_json:
    official_app_list[x['appid']]=x['name']
print(len(full_games_list))
game_df=pd.read_csv(os.path.join(testing_folder,'20190124_games.csv'))
all_categories=set()
all_genres=set()
all_developers={}
all_publishers={}
for x in ['genres','developers','publishers','categories']:
    game_df[x]=game_df[x].apply(ast.literal_eval)
for i,row in game_df.iterrows():
    all_categories=all_categories | set(row['categories'])
    all_genres=all_genres | set(row['genres'])
    #print(all_genres)
    for developer in row['developers']:
        if developer not in all_developers:
            all_developers[developer]=1
        else:
            all_developers[developer]+=1
    for publisher in row['publishers']:
        if publisher not in all_publishers:
            all_publishers[publisher]=1
        else:
            all_publishers[publisher]+=1
#print(all_genres)
all_genres.remove('Early Access')
all_categories=sorted(list(all_categories))
all_genres=sorted(list(all_genres))

4935


In [3]:
num_reviews_df=pd.read_csv(os.path.join(testing_folder,'20190126_num_reviews.csv')).set_index('appid')
early_access_start_time_df=pd.read_csv(os.path.join(testing_folder,'20190126_early_access_start_time.csv')).set_index('appid')
normal_review_start_time_df=pd.read_csv(os.path.join(testing_folder,'20190126_normal_review_start_time.csv')).set_index('appid')
num_english_reviews={}

In [35]:
csv_folder=os.path.join(parent_folder,"bulk","csv_no_text","early_access_standardized")
num_reviews_df_no_index=num_reviews_df.reset_index('appid')

current_ea_first_month_reviewers={}
former_ea_first_month_reviewers={}
for i,row in num_reviews_df_no_index.iterrows():
    appid=row['appid']
    try:
        df=pd.read_csv(os.path.join(csv_folder,str(appid)+'_reviews_all.csv'))
        found_file=True
    except:
        found_file=False
    if found_file:
        reviewer_set=set(df[df['timestamp_created']<early_access_start_time_df.loc[appid]['time']+30*24*60*60].steamid.tolist())
        if appid in former_ea_games:
            for reviewer in reviewer_set:
                if reviewer not in former_ea_first_month_reviewers:
                    former_ea_first_month_reviewers[reviewer]=0
                former_ea_first_month_reviewers[reviewer]+=1
        else:
            for reviewer in reviewer_set:
                if reviewer not in current_ea_first_month_reviewers:
                    current_ea_first_month_reviewers[reviewer]=0
                current_ea_first_month_reviewers[reviewer]+=1
    clear_output(wait=True)
    print(str(i)+"/"+str(len(num_reviews_df_no_index)-1))
print("Done!")

4868/4868
Done!


In [81]:
print(len(former_ea_first_month_reviewers))
print(len(current_ea_first_month_reviewers))
print(len(set(former_ea_first_month_reviewers.keys()) | set(current_ea_first_month_reviewers.keys())))

310598
357860
626797


In [122]:
fe_reviewers_copy=copy.deepcopy(former_ea_first_month_reviewers)
ce_reviewers_copy=copy.deepcopy(current_ea_first_month_reviewers)
print(len(fe_reviewers_copy))
print(len(ce_reviewers_copy))
for key in fe_reviewers_copy:
    if key in ce_reviewers_copy:
        ce_reviewers_copy.pop(key)
print(len(ce_reviewers_copy))

temp_list=sorted(list(fe_reviewers_copy.keys()))
temp_idxs=list(range(len(temp_list)))
randomized_idxs=[temp_idxs.pop(random.randrange(len(temp_idxs))) for x in range(len(temp_list))]
idx_dict={}
randomized_list=[]
for i in range(len(randomized_idxs)):
    randomized_list.append(temp_list[randomized_idxs[i]])
    idx_dict[temp_list[randomized_idxs[i]]]=i

reviewer_friends_dict={'steamid':randomized_list,
                       'num_friends':[-1 for _ in randomized_list],
                       'private':[True for _ in randomized_list],
                       'request_error':[True for _ in randomized_list],
                       'num_games':[-1 for _ in randomized_list],
                       'num_reviews':[-1 for _ in randomized_list],
                       'downloaded':[False for _ in randomized_list]}

temp_list=sorted(list(ce_reviewers_copy.keys()))
temp_idxs=list(range(len(temp_list)))
randomized_idxs=[temp_idxs.pop(random.randrange(len(temp_idxs))) for x in range(len(temp_list))]
ce_idx_dict={}
randomized_list=[]
for i in range(len(randomized_idxs)):
    randomized_list.append(temp_list[randomized_idxs[i]])
    ce_idx_dict[temp_list[randomized_idxs[i]]]=i

ce_reviewer_friends_dict={'steamid':randomized_list,
                       'num_friends':[-1 for _ in randomized_list],
                       'private':[True for _ in randomized_list],
                       'request_error':[True for _ in randomized_list],
                       'num_games':[-1 for _ in randomized_list],
                       'num_reviews':[-1 for _ in randomized_list],
                       'downloaded':[False for _ in randomized_list]}

print("Done!")

310598
357860
316199
Done!


In [124]:
print(len(reviewer_friends_dict['steamid']))
print(len(ce_reviewer_friends_dict['steamid']))

310598
316199


In [125]:
for i,row in num_reviews_df_no_index.iterrows():
    appid=row['appid']
    try:
        df=pd.read_csv(os.path.join(csv_folder,str(appid)+'_reviews_all.csv'))
        found_file=True
    except:
        found_file=False
    if found_file:
        df=df[df['timestamp_created']<early_access_start_time_df.loc[appid]['time']+30*24*60*60]
        for j, reviewer_row in df.iterrows():
            if reviewer_row['steamid'] in idx_dict:
                reviewer_idx=idx_dict[reviewer_row['steamid']]
                reviewer_friends_dict['num_games'][reviewer_idx]=reviewer_row['num_games_owned']
                reviewer_friends_dict['num_reviews'][reviewer_idx]=reviewer_row['num_reviews']
            if reviewer_row['steamid'] in ce_idx_dict:
                ce_reviewer_idx=ce_idx_dict[reviewer_row['steamid']]
                ce_reviewer_friends_dict['num_games'][ce_reviewer_idx]=reviewer_row['num_games_owned']
                ce_reviewer_friends_dict['num_reviews'][ce_reviewer_idx]=reviewer_row['num_reviews']
    clear_output(wait=True)
    print(str(i)+"/"+str(len(num_reviews_df_no_index)-1))
    
print("Done!")

4868/4868
Done!


In [134]:
reviewer_df.head()

,steamid,num_friends,private,request_error,num_games,num_reviews,downloaded
0,76561198080706629,-1,True,True,149,11,False
1,76561198096219905,-1,True,True,9,8,False
2,76561198143214219,-1,True,True,20,2,False
3,76561198101816502,-1,True,True,34,1,False
4,76561198071325976,-1,True,True,54,5,False


In [138]:
out_folder=os.path.join(parent_folder,"bulk","friends_counts")
if not os.path.exists(out_folder):
    os.makedirs(out_folder)

reviewer_df=pd.DataFrame.from_dict(reviewer_friends_dict)
ce_reviewer_df=pd.DataFrame.from_dict(ce_reviewer_friends_dict)
reviewer_df.to_csv(os.path.join(out_folder,'fe_game_reviewers_0.csv'),index=False)
ce_reviewer_df.to_csv(os.path.join(out_folder,'ce_game_reviewers_0.csv'),index=False)
np.savetxt(os.path.join(out_folder,'fe_game_reviewers_list.csv'),np.array(list(fe_reviewers_copy)), delimiter=",")
np.savetxt(os.path.join(out_folder,'ce_game_reviewers_list.csv'),np.array(list(ce_reviewers_copy)), delimiter=",")

In [4]:
out_folder=os.path.join(parent_folder,"bulk","friends_counts")
reviewer_df=pd.read_csv(os.path.join(out_folder,'fe_game_reviewers_68000.csv'))

In [ ]:
html_out_folder=os.path.join(parent_folder,"bulk","friends_html")
if not os.path.exists(html_out_folder):
    os.makedirs(html_out_folder)

hypcurrent=67999
for i,row in reviewer_df.iterrows():
    steamid=int(row['steamid'])
    if i>hypcurrent or reviewer_df.loc[i,'downloaded']==False:
        nonzero_found=False
        try:
            page=requests.get('https://steamcommunity.com/profiles/'+str(steamid)+'/friends').text
            foundpage=True
        except:
            foundpage=False
            num_friends=-1
            private=False
            request_error=True
        if foundpage:
            try:
                with open(os.path.join(html_out_folder,str(i)+'-'+str(steamid)),'w') as file:
                    file.write(page)
            except:
                pass
            test_match=re.search('"cFriends":[0-9]+',page)
            if test_match is not None:
                try:
                    num_friends=int(test_match.group().split(':')[1])
                    if i<hypcurrent and reviewer_df.loc[i,'downloaded']==False:
                        print(str(i)+" corrected!")
                    if num_friends>0:
                        nonzero_found=True
                        request_error=False
                        private=False
                except:
                    pass
            if not nonzero_found:
                try:
                    page=requests.get('https://steamcommunity.com/profiles/'+str(steamid)).text
                    foundpage=True
                except:
                    foundpage=False
                    num_friends=-1
                    private=False
                    request_error=True
                if foundpage:
                    soup = bs4.BeautifulSoup(page, 'html.parser')
                    num_friends=-1
                    private=False
                    request_error=False
                    div = soup.find("div", {"class": "profile_friend_links"})

                    if div is not None:
                        span = div.find("span",{"class": "profile_count_link_total"})
                        if span is not None:
                            try:
                                num_friends=int(span.text.replace(',', ''))
                                if i<hypcurrent and reviewer_df.loc[i,'downloaded']==False:
                                    print(str(i)+" corrected!")
                            except:
                                request_error=True
                        else:
                            #print("A: "+str(steamid))
                            #print("X")
                            #print(div)
                            num_friends=0
                    else:
                        for div in soup.find_all("div",{"class": "profile_count_link ellipsis"}):
                            if div is not None:
                                span=div.find("span",{"class": "count_link_label"})
                                if span is not None:
                                    if span.text=="Reviews":
                                        span=div.find("span",{"class": "profile_count_link_total"})
                                        if span is not None:
                                            #print("A: "+str(steamid))
                                            num_friends=0
                                            #print("Y")
                                            #print(div)
                                            #print("Z")
                                            #print(span)
                        if num_friends==-1:
                            if soup.find("div",{"class": "profile_private_info"}) is not None:
                                private=True
                            else:
                                request_error=True
        reviewer_df.loc[i,'private']=private
        reviewer_df.loc[i,'request_error']=request_error
        if request_error:
            reviewer_df.loc[i,'downloaded']=False
        else:
            reviewer_df.loc[i,'downloaded']=True
        reviewer_df.loc[i,'num_friends']=num_friends
        if i%1000==0:
            reviewer_df.to_csv(os.path.join(out_folder,'fe_game_reviewers_'+str(i)+'.csv'))
        if request_error:
            print("REQUEST ERROR: "+str(i)+": "+str(steamid))
        elif i%100==0:
            print(str(i)+"/"+str(len(reviewer_df)))
        #break
    #else:
    #    break
reviewer_df.to_csv(os.path.join(out_folder,'fe_game_reviewers_'+str(i)+'.csv'))
print("Done!")

REQUEST ERROR: 2021: 76561198230715268
REQUEST ERROR: 6079: 76561198049191329
7558 corrected!
7563 corrected!
7564 corrected!
7565 corrected!
7566 corrected!
7567 corrected!
7568 corrected!
7569 corrected!
7570 corrected!
7571 corrected!
7572 corrected!
7573 corrected!
7574 corrected!
7575 corrected!
7576 corrected!
7577 corrected!
7578 corrected!
7579 corrected!
7580 corrected!
7581 corrected!
7582 corrected!
7583 corrected!
7584 corrected!
7585 corrected!
7586 corrected!
7587 corrected!
7588 corrected!
7589 corrected!
7590 corrected!
7591 corrected!
7592 corrected!
7593 corrected!
7594 corrected!
7595 corrected!
7596 corrected!
7597 corrected!
7598 corrected!
7599 corrected!
7600 corrected!
7600/310598
7601 corrected!
7602 corrected!
7603 corrected!
7604 corrected!
7605 corrected!
7606 corrected!
7607 corrected!
7608 corrected!
7609 corrected!
7610 corrected!
7611 corrected!
7612 corrected!
7613 corrected!
7614 corrected!
7615 corrected!
7616 corrected!
7617 corrected!
7618 corrected

101400/310598
101500/310598
101600/310598
101700/310598
101800/310598
101900/310598
102000/310598
102100/310598
102200/310598
102300/310598
102400/310598
102500/310598
102600/310598
102700/310598
102800/310598
102900/310598
REQUEST ERROR: 102928: 76561198036010259
103000/310598
103100/310598
103200/310598
103300/310598
103400/310598
103500/310598
103600/310598
103700/310598
103800/310598
103900/310598
104000/310598
104100/310598
104200/310598
104300/310598
104400/310598
104500/310598
104600/310598
104700/310598
104800/310598
104900/310598
105000/310598
105100/310598
105200/310598
105300/310598
105400/310598
105500/310598
105600/310598
105700/310598
105800/310598
105900/310598
106000/310598
106100/310598
106200/310598
106300/310598
106400/310598
106500/310598
106600/310598
106700/310598
106800/310598
106900/310598
107000/310598
107100/310598
107200/310598
107300/310598
107400/310598
107500/310598
107600/310598
107700/310598
107800/310598
107900/310598
108000/310598
108100/310598
108200/

155600/310598
155700/310598
155800/310598
155900/310598
156000/310598
156100/310598
156200/310598
156300/310598
156400/310598
156500/310598
156600/310598
156700/310598
156800/310598
156900/310598
157000/310598
157100/310598
157200/310598
157300/310598
157400/310598
157500/310598
157600/310598
157700/310598
157800/310598
157900/310598
158000/310598
158100/310598
158200/310598
158300/310598
158400/310598
158500/310598
158600/310598
158700/310598
158800/310598
158900/310598
159000/310598
159100/310598
159200/310598
159300/310598
159400/310598
159500/310598
159600/310598
159700/310598
159800/310598
159900/310598
160000/310598
160100/310598
160200/310598
160300/310598
160400/310598
160500/310598
160600/310598
160700/310598
160800/310598
160900/310598
161000/310598
161100/310598
161200/310598
161300/310598
161400/310598
161500/310598
161600/310598
161700/310598
161800/310598
161900/310598
162000/310598
162100/310598
162200/310598
162300/310598
162400/310598
162500/310598
REQUEST ERROR: 16260

In [6]:
reviewer_df.head()

,Unnamed: 0,steamid,num_friends,private,request_error,num_games,num_reviews,downloaded
0,0,76561198080706629,87,False,False,149,11,True
1,1,76561198096219905,11,False,False,9,8,True
2,2,76561198143214219,-1,True,False,20,2,True
3,3,76561198101816502,20,False,False,34,1,True
4,4,76561198071325976,0,False,False,54,5,True


In [192]:
print(re.search('"cFriends":adsads[0-9]+',a))

None


In [73]:
print(pd.DataFrame.from_dict({"a":[1,2],"b":[8,9]}))

   a  b
0  1  8
1  2  9


In [63]:
for div in soup.find_all("div",{"class": "profile_count_link ellipsis"}):
    span=div.find("span",{"class": "count_link_label"})
    if span.text=="Reviews":
        span=div.find("span",{"class": "profile_count_link_total"})
        print(int(span.text))

1


In [65]:
print(soup.find("div",{"class": "profile_private_info"}))

None
